In [ ]:
# required for importin modules from other directories
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 


import numpy as np
#from KNNRegressor import KNNRegressor
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler

import config
from common import DataParser
from KNN.KNNRegressor import KNNRegressor

# ChunkViewer

How to

In [ ]:
from math import floor
import numpy as np
from KNNRegressor import ChunkViewer

## Deprecated
def get_chunks(arr:np.ndarray, chunk_size:int):
    chunk_size = chunk_size
    samples = arr.shape[0]

    if chunk_size > samples:
        print("Can do it all in one chunk!")
        chunk_size = samples

    print(f"chunks in samples: floor({samples}/{chunk_size})={floor(samples/chunk_size)}")

    chunks = [(x*chunk_size, x*chunk_size+chunk_size) for x in range(floor(samples/chunk_size))]
    remainder = samples%chunk_size
    if remainder:
        chunks += [(chunks[-1][1], chunks[-1][1] + remainder)]
    return chunks

In [ ]:
arr = np.zeros(10)
chunks = get_chunks(arr, 2)
chunks
for chunk in chunks:
    print(chunk)

In [ ]:
arr = np.zeros(50).reshape(10,5)
arr[2:4,:] = 1
arr[4:6,:] = 2
arr[6:8,:] = 3
arr[8:,:] = 4

arr

In [ ]:
np.array_split(arr, 3)

In [ ]:
viewer = ChunkViewer(4, 10)
[x for x in viewer]

#[x for x in viewer]

In [ ]:
viewer = ChunkViewer(4, 10)
[x for x in viewer.generator()]

In [ ]:
viewer = ChunkViewer(4, 10)
for x in viewer.generator(arr):
    print(x)

# KNNRegressor

In [ ]:
samples, values = [[0., 0., 0.], [0., .5, 0.], [1., 1., .5]], [0, 0.5, 2.5]
neigh = KNNRegressor(n_neighbors=2, p=1,debug=True)
neigh.fit(samples, values)
result = neigh.predict(np.array([[0, 0., .1]]))
print()
should_be = 0.25
print(f"Result: (should be {should_be})")
print(f"{result} -> {result==should_be}")

# House Prices Test

In [ ]:
class Test:
    def __init__(self, val=1):
        self._att = val
    
    @property
    def att(self):
        return self._att

    @att.setter
    def att(self, val):
        self._att = val

a = Test()
print(a.att)
a.att = 2
a.att

In [ ]:
x_train, y_train = DataParser.parse_test_housePrices(splitData=True)

k = 3
p = 1
weights = "uniform" #"distance"
reg = KNNRegressor(n_neighbors=k, p=p, debug=False)
reg.fit(x_train, y_train)
display(reg)

print("Predict")
y_pred, n = reg.predict(x_train.iloc[0:3,:].to_numpy(), ret_distances=2)
display(y_pred, n)

print("Make this look like the above^")
reg.chunk_size = 2
y_pred = reg.predict(x_train.iloc[0:3,:].to_numpy())
display(y_pred)

print("Nice")


# Moneyball

In [ ]:
df = DataParser.parse_moneyball(splitData=False)

In [ ]:
numeric_features = ["RS", "RA", "OBP", "SLG", "BA", "Playoffs", "G", "OOBP", "OSLG"]

In [ ]:
df = df[numeric_features + ["W"]]
df.dropna(inplace=True)
x = df[numeric_features]
y = df["W"]
x, y

In [ ]:
def split_XY(x, y):
    x_train, x_split, y_train, y_split = train_test_split(x, y, test_size=0.4)
    x_valid, x_test, y_valid, y_test = train_test_split(x_split, y_split, test_size=0.5)
    return x_train, x_valid, x_test, y_train, y_valid, y_test

def split_df(df):
    train, split = train_test_split(df, test_size=0.4)
    valid, test = train_test_split(split, test_size=0.5)
    return y_train, y_valid, y_test

def preprocess(df, scaler=None, ret_df=True):
    numeric_features = ["RS", "RA", "OBP", "SLG", "BA", "Playoffs", "G", "OOBP", "OSLG"]
    df = df[numeric_features + ["W"]]
    df.dropna(inplace=True)
    x = df[numeric_features]
    if scaler:
        x = scaler.fit_transform(x)
    y = df["W"]
    return x, y, df

def get_time(start, scale=1e-9):
    return (timer() - start ) * scale

In [ ]:
from time import time_ns as timer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor as skKNN
from sklearn.model_selection import train_test_split
import pandas as pd


COLS = ["name", "param", "r2", "mse", "train_time", "inference_time"]

def experiment(df, models, N=[1], scaler=None):
    x, y, df = preprocess(df, scaler, ret_df=True)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    results = pd.DataFrame(columns=COLS)
    plt.figure(figsize=(10,8))

    for n in N:
        n_train = int(n*len(y_train))
        n_test = int(n*len(y_test))
        x_train_sub, y_train_sub = x_train.iloc[:n_train,:], y_train.iloc[:n_train]
        x_test_sub, y_test_sub = x_test.iloc[:n_test,:], y_test.iloc[:n_test]
        for name, model in models.items():
            params = model.get_params()
            print(name)
            #display(model)
            
            train_time = timer()
            model.fit(x_train_sub, y_train_sub)
            train_time = get_time(train_time)

            inference_time = timer()
            predictions = model.predict(x_test_sub)
            inference_time = get_time(inference_time)

            r2 = r2_score(y_test_sub, predictions)
            mse = mean_squared_error(y_test_sub, predictions)

            result = {
                "name": name,
                "N": n_train+n_test,
                "n_train": n_train,
                "n_test": n_test,
                "param": params["n_neighbors"],
                "r2": r2,
                "mse": mse,
                "train_time": train_time,
                "inference_time": inference_time
            }
            results = results.append(result, ignore_index=True)
            #display(result)

            marker = "x" if name=="my_knn" else "p"
            ls = "--" if name=="my_knn" else "-."
            if n == N[-1]:
                plt.plot(range(len(predictions)), predictions, linestyle=ls, marker=marker, markersize=5, label=name);
        if n == N[-1]:
            plt.plot(range(len(y_test)), y_test, linestyle="", marker="o", markersize=8, label="y_valid");
            plt.legend()
    return results

In [ ]:
models = []

reg = KNNRegressor()
reg2 = KNNRegressor()
reg2.chunk_size = 20
reg3 = KNNRegressor()
reg3.chunk_size = 200
reg4 = KNNRegressor()
reg4.chunk_size = 400

knn = skKNN(algorithm="brute")
knn2 = skKNN(algorithm="kd_tree")
knn3 = skKNN(algorithm="ball_tree")

models = {"my_knn": reg, f"my_knn_chunked_{reg2.chunk_size}": reg2, f"my_knn_chunked_{reg3.chunk_size}": reg3, f"my_knn_chunked_{reg4.chunk_size}": reg4, "sk_knn_brute": knn, "sk_knn_kd": knn2, "sk_knn_ball": knn3}
for x in models:
    display(str(x)+": ")
    display(models[x].get_params())
    if not x == "sk_knn_ball":
        print("_"*30)


In [ ]:
scaler = None #StandardScaler()
N = [x for x in np.arange(0.2, 1.1, 0.2)]
print(N)
results = experiment(df, models, N, scaler=scaler)

In [ ]:
results

In [ ]:
MODEL = KNeighborsRegressor
params = {
    "weights" : ["uniform"],
    "algorithm" : ["brute", "kd_tree", "ball_tree"]
    }
NAME = "KNN"
PATH = "out/"+NAME+"/"

modeltrainer = ModelTrainer(MODEL, params, X, Y, f_eval=r2_score)
########### train with TrainTestSplit  ###################
#modeltrainer.TTSplit(perc = 0.3)
#modeltrainer.train()
#res = modeltrainer.result
#display(res)


########### k-Fold Cross validation  ###################

results_sklknn = modeltrainer.CV_shuffle_split(k = 6, test_size=0.3, random_state=42)
#modeltrainer.CV_fold(k = 6)
#modeltrainer.train()
#res = modeltrainer.result
display(results_sklknn)

In [ ]:
MODEL = KNNRegressor
params = {
    "weights": ["uniform"], 
    "chunk_size": [200, 500, 1000]
    }
MODEL()
subset_size = 1000#X.shape[0]
modeltrainer = ModelTrainer(MODEL, params, X[:subset_size, ...], Y.iloc[:subset_size], thread_cnt=4)

########### k-Fold Cross validation  ###################
results_myknn = modeltrainer.CV_shuffle_split(k=6, test_size=0.3, random_state=42)
#modeltrainer.train()